# Neo4j에 그래프 로드

In [ ]:
import os
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"

In [ ]:
import pickle

with open('samples/graph_docs.pkl', 'rb') as f:
    loaded_graph_docs = pickle.load(f)

In [ ]:
from langchain.graphs import Neo4jGraph
from py2neo import Graph

py_graph = Graph()
n4j_graph = Neo4jGraph()
vector_search_enabled = True
index_name = "content_embedding_index"

In [ ]:
def init_graph(vector_search_enabled: bool):
    py_graph.run("MATCH (n) DETACH DELETE n")

    if vector_search_enabled:
        query = """
            CREATE VECTOR INDEX $index_name IF NOT EXISTS
            FOR (n:Content) ON (n.embedding)
            OPTIONS {
                indexConfig: {
                    `vector.dimensions`: 1024,
                    `vector.similarity_function`: 'cosine'
                }
            }
        """
        params = {"index_name": index_name}
        py_graph.run(query, params)

    py_graph.run("""
        CREATE FULLTEXT INDEX Search_Content_by_FullText IF NOT EXISTS
        FOR (n:Content) 
        ON EACH [n.text] 
        OPTIONS { indexConfig: { `fulltext.analyzer`: "english"}}
        """)

init_graph(vector_search_enabled)

n4j_graph.add_graph_documents(
  loaded_graph_docs, 
  baseEntityLabel=True
)

### Vector Search (Global)

In [ ]:
from langchain_aws import BedrockEmbeddings
from langchain.vectorstores import Neo4jVector

if vector_search_enabled == True:
    embeddings = BedrockEmbeddings(model_id="cohere.embed-multilingual-v3", region_name="us-east-1")

    vector_store = Neo4jVector.from_existing_index(
        embedding=embeddings,
        index_name=index_name,
        node_label="Content",
        text_node_property="text", 
        embedding_node_property="embedding"
    )

    question = "Bedrock의 Agent에서 API 스키마를 설정하는 방법"    
    question_embedding = embeddings.embed_query(question)
    k=5

    cypher_query = """
        CALL db.index.vector.queryNodes($index_name, $k, $question_embedding) YIELD node, score
        WITH DISTINCT node, score
        WHERE node:Content
        RETURN node.text AS text, score
        ORDER BY score DESC
    """

    params = {
        "index_name": index_name,
        "question_embedding": question_embedding,
        "k": k
    }

    results = vector_store.query(cypher_query, params=params)

    for result in results:
        print("=================\n")
        print(result['text'])
        print("=================\n")

### Vector Search (Node Level)

In [ ]:
if vector_search_enabled == True:

    vector_store = Neo4jVector.from_existing_index(
        embedding=embeddings,
        index_name=index_name,
        node_label="Content",
        text_node_property="text", 
        embedding_node_property="embedding"
    )
    
    find_parent_query = """
    MATCH (parent:Title)
    WHERE parent.value = $parent_value
    RETURN id(parent) AS parent_id
    """

    parent_params = {
        "parent_value": "Define OpenAPI schemas for your agent's action groups in Amazon Bedrock"
    }

    parent_result = vector_store.query(find_parent_query, params=parent_params)
    parent_id = parent_result[0]['parent_id'] if parent_result else None

    print("parent_id:", parent_id)
    if parent_id is None:
        print("Parent node not found")
    else:
        embeddings = BedrockEmbeddings(model_id="cohere.embed-multilingual-v3", region_name="us-east-1")

        question = "Bedrock의 Agent에서 API 스키마를 설정하는 방법"
        question_embedding = embeddings.embed_query(question)
        k=5
        cypher_query = """
            MATCH (parent)-[:HAS_CONTENTS]->(child:Content)
            WHERE id(parent) = $parent_id
            WITH child
            CALL db.index.vector.queryNodes($index_name, $k, $question_embedding) YIELD node, score
            WHERE node = child
            RETURN id(node) AS node_id, node.text AS text, score, {} AS metadata
            ORDER BY score DESC
            LIMIT $k
        """

        params = {
            "parent_id": parent_id,
            "question_embedding": question_embedding,
            "k": k,
            "index_name": index_name
        }

        results = vector_store.query(cypher_query, params=params)

        for result in results:
            print("=================\n")
            print(f"Node ID: {result['node_id']}")
            print(f"Text: {result['text']}")
            print("=================\n")



### Vector Search (SubGraph Level)

In [ ]:
if vector_search_enabled == True:

    vector_store = Neo4jVector.from_existing_index(
        embedding=embeddings,
        index_name=index_name,
        node_label="Content",
        text_node_property="text", 
        embedding_node_property="embedding"
    )
    
    embeddings = BedrockEmbeddings(model_id="cohere.embed-multilingual-v3", region_name="us-east-1")

    question = "Bedrock의 Agent에서 API 스키마를 설정하는 방법"
    question_embedding = embeddings.embed_query(question)
    k=5

    subgraph = "Amazon Bedrock"
    cypher_query = """
        MATCH (root:Title {level: "1", value: $subgraph})
        MATCH (root)-[:HAS_CHILD*0..]->(title:Title)-[:HAS_CONTENTS]->(content:Content)

        CALL db.index.vector.queryNodes($index_name, $k, $question_embedding) YIELD node, score
        WHERE node = content

        RETURN node.text AS text, score
        ORDER BY score DESC
        LIMIT $k
    """

    params = {
        "subgraph": subgraph,
        "question_embedding": question_embedding,
        "k": k,
        "index_name": index_name
    }

    results = vector_store.query(cypher_query, params=params)

    for result in results:
        print("=================\n")
        print(f"Text: {result['text']}")
        print("=================\n")

